In [9]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [17]:
import requests
import json
import time
import os
import math

In [11]:
secrets_file = "secrets.json"

if not os.path.exists(secrets_file):
    raise FileNotFoundError(f"{secrets_file} not found. Please create the file with your API key.")

with open(secrets_file, "r") as file:
    secrets = json.load(file)

API_KEY = secrets.get("google_api_key")

if not API_KEY:
    raise ValueError("API key not found in secrets.json. Please add your key under 'google_api_key'.")

In [12]:
BASE_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


In [21]:
# Bounding box for Calgary (approximate)
# Southwest corner: 50.8429, -114.3217
# Northeast corner: 51.1895, -113.8573
SW_LAT, SW_LNG = 50.8429, -114.3217
NE_LAT, NE_LNG = 51.1895, -113.8573

# length: 38.52km=~39km, width: 36.20km
LENGTH = 39 #km
WIDTH = 37 # km
GRID_SIZE = 0.5 # km

GRID_DIV_LENGTH = LENGTH  / GRID_SIZE
GRID_DIV_WIDTH = WIDTH  / GRID_SIZE
# Separate grid into 500m x 500m grids
# Grid division parameters
GRID_DIVISIONS = 78 # Number of divisions in each direction
RADIUS = math.ceil(math.sqrt(2*(float(GRID_SIZE) * 1000 / 2)**2))   # Search radius in meters for each grid cell

# Generate grid of center points
lat_step = (NE_LAT - SW_LAT) / GRID_DIV_WIDTH
lng_step = (NE_LNG - SW_LNG) / GRID_DIV_LENGTH
grid_centers = [
    (SW_LAT + i * lat_step, SW_LNG + j * lng_step)
    for i in range(GRID_DIVISIONS)
    for j in range(GRID_DIVISIONS)
]

print(RADIUS)
print(len(grid_centers))

SyntaxError: '(' was never closed (3604188863.py, line 17)

In [14]:
def fetch_results(url, params):
    """
    Fetches data from the given URL with provided parameters.
    """
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response}")
        return None

In [15]:
# Collect all place IDs
all_place_ids = set()
all_places = []
queries = 0
grid = 0
for center in grid_centers:

    print(f"Querying center: {center}")
    params = {
        "location": f"{center[0]},{center[1]}",
        "radius": RADIUS,
        "type": "restaurant",
        "key": API_KEY
    }
    grid += 1
    next_page_token = None
    while True:
        if next_page_token:
            params["pagetoken"] = next_page_token
            time.sleep(2)  # Delay as per Google API guidelines

        data = fetch_results(BASE_URL, params)
        queries += 1
        print("Grid Number: ", grid)
        print("Query Number: ", queries)

        if data['status'] == 'OK':
            all_places.extend(data['results'])

            for place in data["results"]:
                all_place_ids.add(place["place_id"])

            output_file = "resturaunt_ids.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(list(all_place_ids), f, indent=4)

            output_file = "resturaunts_details.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(list(all_places), f, indent=4)

            if "next_page_token" in data:
                next_page_token = data.get("next_page_token")
            else:
                break
        else:
            break

Querying center: (50.8429, -114.3217)
Grid Number:  1
Query Number:  1
Querying center: (50.8429, -114.30622000000001)
Grid Number:  2
Query Number:  2
Querying center: (50.8429, -114.29074)
Grid Number:  3
Query Number:  3
Querying center: (50.8429, -114.27526)
Grid Number:  4
Query Number:  4
Querying center: (50.8429, -114.25978)
Grid Number:  5
Query Number:  5
Querying center: (50.8429, -114.24430000000001)
Grid Number:  6
Query Number:  6
Querying center: (50.8429, -114.22882)
Grid Number:  7
Query Number:  7
Querying center: (50.8429, -114.21334)
Grid Number:  8
Query Number:  8
Querying center: (50.8429, -114.19786)
Grid Number:  9
Query Number:  9
Querying center: (50.8429, -114.18238000000001)
Grid Number:  10
Query Number:  10
Querying center: (50.8429, -114.1669)
Grid Number:  11
Query Number:  11
Querying center: (50.8429, -114.15142)
Grid Number:  12
Query Number:  12
Querying center: (50.8429, -114.13594)
Grid Number:  13
Query Number:  13
Querying center: (50.8429, -114

In [33]:
output_file = "resturaunt_ids.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(list(all_place_ids), f, indent=4)

output_file = "resturaunts_details.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(list(all_places), f, indent=4)